In [ ]:
!pip install spacy pdfplumber docx2txt scikit-learn
!python -m spacy download en_core_web_sm

In [ ]:
!pip install docx2txt
!pip install pdfplumber

In [ ]:
import spacy, pdfplumber, docx2txt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def extract_text(file_path):
    #Extracts text from a PDF or DOCX file
    if file_path.endswith(".pdf"):
        with pdfplumber.open(file_path) as pdf:
            return " ".join([page.extract_text() for page in pdf.pages if page.extract_text()])
    elif file_path.endswith(".docx"):
        return docx2txt.process(file_path)
    return ""

def process_resume(text):
   # """Analyzes resume text and extracts key information like skills, education, and experience."""
    doc = nlp(text)
    return {ent.label_: ent.text for ent in doc.ents}

def match_job(resume_text, job_desc):
   # """Calculates a match score between the resume and job description using TF-IDF and cosine similarity."""
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([resume_text, job_desc])
    return round(cosine_similarity(vectors[0], vectors[1])[0][0] * 100, 2)

if __name__ == "__main__":
    nlp = spacy.load("en_core_web_sm")
    file_path = input("Enter resume file path (PDF/DOCX): ").strip()
    job_desc = input("Enter job description: ").strip()
    
    resume_text = extract_text(file_path)
    resume_info = process_resume(resume_text)
    match_score = match_job(resume_text, job_desc)
    
    print("\n Resume Details:")
    for label, value in resume_info.items():
        print(f"{label}: {value}")
    
    print(f"\n Job Match Score: {match_score}%")
